In [1]:
stock = 'AAPL'
days_back = 100

In [2]:
import pandas as pd                              # 表格和数据处理
import matplotlib.pyplot as plt                  # 绘图
import seaborn as sns                            # 更多绘图功能
import pandas_datareader.data as webdata

from itertools import product                    # 一些有用的函数
from tqdm import tqdm_notebook

import datetime                                  # 日期函数

import warnings                                  # 勿扰模式
warnings.filterwarnings('ignore')

%matplotlib inline

In [3]:
now = datetime.date.today()
start = now - datetime.timedelta(days=days_back)
# df = data.DataReader(stock, "iex", start=start, end=end)
df = webdata.get_data_yahoo(stock, start=start, end=now)['Adj Close']

In [4]:
df

Date
2018-07-20    190.770950
2018-07-23    190.940353
2018-07-24    192.325485
2018-07-25    194.139145
2018-07-26    193.531265
2018-07-27    190.312546
2018-07-30    189.246292
2018-07-31    189.624954
2018-08-01    200.795792
2018-08-02    206.665207
2018-08-03    207.263107
2018-08-06    208.339340
2018-08-07    206.386185
2018-08-08    206.525696
2018-08-09    208.149994
2018-08-10    207.529999
2018-08-13    208.869995
2018-08-14    209.750000
2018-08-15    210.240005
2018-08-16    213.320007
2018-08-17    217.580002
2018-08-20    215.460007
2018-08-21    215.039993
2018-08-22    215.050003
2018-08-23    215.490005
2018-08-24    216.160004
2018-08-27    217.940002
2018-08-28    219.699997
2018-08-29    222.979996
2018-08-30    225.029999
                 ...    
2018-09-17    217.880005
2018-09-18    218.240005
2018-09-19    218.369995
2018-09-20    220.029999
2018-09-21    217.660004
2018-09-24    220.789993
2018-09-25    222.190002
2018-09-26    220.419998
2018-09-27    224.94

In [5]:
# 需要预测的5天日期
d = [None] * 5
j = 0

for i in range(0,7):
    if (now + datetime.timedelta(i)).weekday() < 5:
        d[j] = str(now + datetime.timedelta(i))
        j = j + 1

In [6]:
#时序数据上的线性模型
data = pd.DataFrame(df.copy().astype(float))
data.columns = ["y"]

df = pd.DataFrame({'y':[0,0,0,0,0]},
                   index = d)
data = data.append(df)

for i in range(6, 25):
    data["lag_{}".format(i)] = data.y.shift(i)

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

y = data.dropna().y # 删除n/a的数据
X = data.dropna().drop(['y'], axis=1)

# 除去需要预测的5天
train_index = int(len(X)) - 5
X_train = X.iloc[:train_index]
y_train = y.iloc[:train_index]

# 机器学习
lr = LinearRegression()
lr.fit(X_train, y_train)
prediction = lr.predict(X.tail(5))
precentage = prediction.mean()
print(prediction.mean())
print(prediction)
# plt.figure(figsize=(15, 7))
# plt.plot(prediction, "g", label="prediction", linewidth=2.0)


228.40285006552708
[232.45812349 227.74305803 223.40490637 229.07408759 229.33407484]


In [7]:
#储存数据到excel
from openpyxl import load_workbook
df = pd.DataFrame(prediction)
df = df.values.T.tolist()
stock_name = [str(stock)]
wb = load_workbook('/Users/BartSu/Desktop/StockRS_Result.xlsx')
ws = wb.worksheets[0]

ws.append(stock_name)

# Append 2 new Rows - Columns A - D
for row_data in df:
    # Append Row Values
    ws.append(row_data)

wb.save("/Users/BartSu/Desktop/StockRS_Result.xlsx")